##### Conectamos a una base de datos SQLite llamada "bdejemplo.db" y creamos varias tablas (Usuarios, Ordenes, Articulos, Cursos, Profesores, Estudiantes, Inscripciones, Aulas)

In [1]:
import sqlite3

# Conectar a la base de datos SQLite (o crearla si no existe)
conn = sqlite3.connect('bdejemplo.db')
cursor = conn.cursor()

# Crear tablas
cursor.execute('''
CREATE TABLE IF NOT EXISTS Usuarios (
    ID INTEGER PRIMARY KEY,
    Nombre TEXT,
    Email TEXT UNIQUE
)
''')

cursor.execute('''
CREATE TABLE IF NOT EXISTS Ordenes (
    OrdenID INTEGER PRIMARY KEY,
    UsuarioID INTEGER,
    Fecha DATE,
    FOREIGN KEY (UsuarioID) REFERENCES Usuarios(ID)
)
''')

cursor.execute('''
CREATE TABLE IF NOT EXISTS Articulos (
    ID INTEGER PRIMARY KEY,
    Titulo TEXT,
    Contenido TEXT
)
''')

cursor.execute('''
CREATE TABLE IF NOT EXISTS Cursos (
    CursoID INTEGER PRIMARY KEY,
    Curso TEXT
)
''')

cursor.execute('''
CREATE TABLE IF NOT EXISTS Profesores (
    ProfesorID INTEGER PRIMARY KEY,
    ProfesorNombre TEXT
)
''')

cursor.execute('''
CREATE TABLE IF NOT EXISTS Estudiantes (
    EstudianteID INTEGER PRIMARY KEY,
    Nombre TEXT
)
''')

cursor.execute('''
CREATE TABLE IF NOT EXISTS Inscripciones (
    EstudianteID INTEGER,
    CursoID INTEGER,
    FOREIGN KEY (EstudianteID) REFERENCES Estudiantes(EstudianteID),
    FOREIGN KEY (CursoID) REFERENCES Cursos(CursoID)
)
''')

cursor.execute('''
CREATE TABLE IF NOT EXISTS Aulas (
    Aula TEXT PRIMARY KEY,
    ProfesorID INTEGER,
    FOREIGN KEY (ProfesorID) REFERENCES Profesores(ProfesorID)
)
''')


##### Insertamos datos en las tablas que creamos anteriormente. Usamos INSERT OR IGNORE para evitar errores si los datos ya existen.

In [2]:
# Insertar datos en las tablas
cursor.execute("INSERT OR IGNORE INTO Usuarios (ID, Nombre, Email) VALUES (1, 'Juan Pérez', 'juan@example.com')")
cursor.execute("INSERT OR IGNORE INTO Usuarios (ID, Nombre, Email) VALUES (2, 'Ana López', 'ana@example.com')")
cursor.execute("INSERT OR IGNORE INTO Usuarios (ID, Nombre, Email) VALUES (3, 'Carlos Ruiz', 'carlos@example.com')")

cursor.execute("INSERT OR IGNORE INTO Ordenes (OrdenID, UsuarioID, Fecha) VALUES (1, 1, '2024-07-01')")
cursor.execute("INSERT OR IGNORE INTO Ordenes (OrdenID, UsuarioID, Fecha) VALUES (2, 2, '2024-07-02')")
cursor.execute("INSERT OR IGNORE INTO Ordenes (OrdenID, UsuarioID, Fecha) VALUES (3, 3, '2024-07-03')")

cursor.execute("INSERT OR IGNORE INTO Articulos (ID, Titulo, Contenido) VALUES (1, 'Artículo 1', 'Contenido del artículo 1')")
cursor.execute("INSERT OR IGNORE INTO Articulos (ID, Titulo, Contenido) VALUES (2, 'Artículo 2', 'Contenido del artículo 2')")
cursor.execute("INSERT OR IGNORE INTO Articulos (ID, Titulo, Contenido) VALUES (3, 'Artículo 3', 'Contenido del artículo 3')")

cursor.execute("INSERT OR IGNORE INTO Cursos (CursoID, Curso) VALUES (1, 'Matemáticas')")
cursor.execute("INSERT OR IGNORE INTO Cursos (CursoID, Curso) VALUES (2, 'Física')")
cursor.execute("INSERT OR IGNORE INTO Cursos (CursoID, Curso) VALUES (3, 'Química')")
cursor.execute("INSERT OR IGNORE INTO Cursos (CursoID, Curso) VALUES (4, 'Biología')")

cursor.execute("INSERT OR IGNORE INTO Profesores (ProfesorID, ProfesorNombre) VALUES (101, 'Lc. García')")
cursor.execute("INSERT OR IGNORE INTO Profesores (ProfesorID, ProfesorNombre) VALUES (102, 'Lc. López')")
cursor.execute("INSERT OR IGNORE INTO Profesores (ProfesorID, ProfesorNombre) VALUES (103, 'Lc. Martínez')")
cursor.execute("INSERT OR IGNORE INTO Profesores (ProfesorID, ProfesorNombre) VALUES (104, 'Lc. Fernández')")

cursor.execute("INSERT OR IGNORE INTO Estudiantes (EstudianteID, Nombre) VALUES (1, 'Juan Pérez')")
cursor.execute("INSERT OR IGNORE INTO Estudiantes (EstudianteID, Nombre) VALUES (2, 'Ana López')")

cursor.execute("INSERT OR IGNORE INTO Inscripciones (EstudianteID, CursoID) VALUES (1, 1)")
cursor.execute("INSERT OR IGNORE INTO Inscripciones (EstudianteID, CursoID) VALUES (1, 2)")
cursor.execute("INSERT OR IGNORE INTO Inscripciones (EstudianteID, CursoID) VALUES (2, 2)")

cursor.execute("INSERT OR IGNORE INTO Aulas (Aula, ProfesorID) VALUES ('101', 101)")
cursor.execute("INSERT OR IGNORE INTO Aulas (Aula, ProfesorID) VALUES ('102', 102)")

##### Usamos EXPLAIN QUERY PLAN para ver el plan de ejecución de la consulta antes y después de crear el índice. Esto muestra cómo la base de datos planea ejecutar la consulta y cómo los índices afectan este plan.

##### Sin el Indice la base de datos realiza un escaneo completo de la tabla Usuarios

In [3]:
# Consulta con EXPLAIN antes de crear el índice
print("EXPLAIN sin índice:")
for row in cursor.execute("EXPLAIN QUERY PLAN SELECT * FROM Usuarios WHERE Nombre = 'Juan Pérez'"):
    print(row)

print("\nConsulta sin índice:")
for row in cursor.execute("SELECT * FROM Usuarios WHERE Nombre = 'Juan Pérez'"):
    print(row)

EXPLAIN sin índice:
(2, 0, 0, 'SCAN Usuarios')

Consulta sin índice:
(1, 'Juan Pérez', 'juan@example.com')


##### Con el Indice la base de datos utiliza el índice idx_nombre para encontrar rápidamente las filas que coinciden con Nombre = 'Juan Pérez'.

In [5]:
# Crear índices
cursor.execute("CREATE INDEX IF NOT EXISTS idx_usuario_fecha ON Ordenes (UsuarioID, Fecha)")
cursor.execute("CREATE INDEX IF NOT EXISTS idx_nombre ON Usuarios (Nombre)")

# Consulta con EXPLAIN despues de crear el índice

print("\nEXPLAIN con índice:")
for row in cursor.execute("EXPLAIN QUERY PLAN SELECT * FROM Usuarios WHERE Nombre = 'Juan Pérez'"):
    print(row)

print("\nConsulta con índice:")
for row in cursor.execute("SELECT * FROM Usuarios WHERE Nombre = 'Juan Pérez'"):
    print(row)

conn.commit()
conn.close()


EXPLAIN con índice:
(3, 0, 0, 'SEARCH Usuarios USING INDEX idx_nombre (Nombre=?)')

Consulta con índice:
(1, 'Juan Pérez', 'juan@example.com')
